In [1]:
from _Setup import *

In [20]:
#train = pd.read_csv(r"C:\Users\Marin\OneDrive\Documents\GitHub\BC-Fall2024-CaseCompetition-Team2\Code\ttrain.csv",
                        low_memory = False)
#test_data = pd.read_csv(r"C:\Users\Marin\OneDrive\Documents\GitHub\BC-Fall2024-CaseCompetition-Team2\Code\ttest.csv", 
                       #low_memory = False)

In [21]:
train = pd.read_csv(missForestTrainFile)
test = pd.read_csv(missForestTestFile)
submission_data = pd.read_csv(rawSubmissionFile)

In [22]:
#Separating columns 
#X = train.drop(columns=['Unit2','Unnamed: 0', 'HospAdmTime','SepsisLabel', 'time']) 
X = train.drop(columns=['Unit2', 'HospAdmTime', 'time'])  
y = train['SepsisLabel']

In [23]:
X.head()

,patient,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,HCO3,...,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,ICULOS,SepsisLabel
0,16426,82.932781,97.792553,36.985761,121.833475,79.608298,62.022952,18.326606,32.976494,23.999359,...,30.287827,10.300167,32.300104,10.298233,247.980239,180.000204,57.03,1.0,1.0,0.0
1,16426,78.000000,99.500000,35.250000,108.500000,84.500000,69.000000,12.000000,31.995111,23.000000,...,34.000000,12.000000,31.245564,14.200000,249.177795,190.403609,57.03,1.0,2.0,0.0
2,16426,80.000000,99.000000,36.200000,113.000000,87.000000,73.000000,12.000000,32.313429,23.999359,...,30.273045,10.299940,32.316444,10.298940,247.980239,179.634235,57.03,1.0,3.0,0.0
3,16426,79.000000,100.000000,36.500000,112.000000,83.000000,68.000000,12.000000,32.680381,24.145191,...,30.439102,10.301595,31.647337,9.075542,247.975450,182.216217,57.03,1.0,4.0,0.0
4,16426,73.000000,100.000000,36.500000,115.000000,80.000000,64.000000,11.000000,32.241118,23.999359,...,30.277911,10.299940,32.365331,10.258271,247.980239,179.924815,57.03,1.0,5.0,0.0


In [5]:
#k = 10  # Adjust this number based on your needs
#selector = SelectKBest(score_func=f_classif, k=k)
#X_selected = selector.fit_transform(X, y)

# Get the names of the selected features
#selected_features = X.columns[selector.get_support()]
#print("Selected Features:", selected_features)

In [15]:
# take a sample from the data 
sampled_patient_ids = X['patient'].drop_duplicates().sample(frac=0.1, random_state=42) 

# Filter the original DataFrame to include only the sampled patient IDs
sampled_data = X[X['patient'].isin(sampled_patient_ids)]
y = sampled_data['SepsisLabel']

In [16]:
selected_columns = ['ICULOS']
X_selected = sampled_data[selected_columns]

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear') 
model.fit(X_selected, y)

In [ ]:
y_pred = model.predict(test)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
}

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_selected, y)

print(f"Best parameters: {grid_search.best_params_}")

In [ ]:
submission_data['SepsisLabel'] = y_pred

# Optionally, save the modified DataFrame to a CSV file
# TODO : create a variable for this path in _Setup.py
# submission_data.to_csv('submission_data_with_predictions.csv', index=False)

# Display the updated DataFrame (optional)
print(submission_data.head())

In [ ]:
num_rows = submission_data.shape[0]
print("Number of rows in submission_data:", num_rows)

In [ ]:
print("\nClassification Report (Training):\n", classification_report(y, y_pred))

In [ ]:
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

## Logistic regression 

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [24]:
#columns_t = ['ICULOS', 'HR', 'O2Sat', 'Temp', 'Resp', 'BaseExcess', 'BUN', 'FiO2', 'Magnesium', 'Age', 'WBC', 'Platelets' , 'Calcium', 'Creatinine', 
            #'MAP', 'SBP' , 'Hct', 'PaCO2']
#columns_test = ['ICULOS', 'HR', 'O2Sat', 'Temp', 'Resp', 'BaseExcess', 'BUN', 'FiO2', 'Magnesium', 'Age', 'WBC', 'Platelets' , 'Calcium', 'Creatinine', 
            #'MAP', 'SBP' , 'Hct', 'PaCO2']

#x_train = train[columns_t]
#x_test = test[columns_test]

In [25]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

x_train_sm = sm.add_constant(X)
model = sm.Logit(y, x_train_sm)
result = model.fit()
y_pred_prob = result.predict(x_train_sm)  
y_pred = (y_pred_prob >= 0.4).astype(int)  

# Calculate the confusion matrix
cm_train = confusion_matrix(y, y_pred)


# Print the summary
print(result.summary())

C:\Users\Marin\anaconda3\envs\sepsis\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\Marin\anaconda3\envs\sepsis\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\Marin\anaconda3\envs\sepsis\Lib\site-packages\statsmodels\discrete\discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

In [ ]:
cm_train = confusion_matrix(y, y_pred)

# Print the confusion matrix
print("Confusion Matrix (Training Data):")
print(cm_train)

In [65]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score

f1 = f1_score(y, y_pred)
print(f"F1 Score (Training Data): {f1:.2f}")

F1 Score (Training Data): 0.33


In [57]:
submission_data['SepsisLabel'] = y_pred

# Optionally, save the modified DataFrame to a CSV file
# TODO : create a variable for this path in _Setup.py
# submission_data.to_csv('submission_data_with_predictions.csv', index=False)

# Display the updated DataFrame (optional)
print(submission_data.head())

       ID  SepsisLabel
0  6560_1            0
1  6560_2            0
2  6560_3            0
3  6560_4            0
4  6560_5            0


In [60]:
num_rows = submission_data.shape[0]
print("Number of rows in submission_data:", num_rows)

Number of rows in submission_data: 464354


In [61]:
submission_data.to_csv('submission_data_Logit.csv', index=False)

## Random Forest 

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_selected, y)

# Make predictions on the test set
y_pred = model.predict(test)

# Evaluate the model
cm = confusion_matrix(y, y_pred)
print("Confusion Matrix (Test Data):")
print(cm)

# Print metrics
accuracy = accuracy_score(y, y_pred)
f1 = f1_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")

## Generalized Estimating Equations 